In [6]:
# Activity 5.01: Clustering Sales Data Using K-Means
# Author: Shabd Mishra

import pandas as pd

# Load dataset
file_url = "https://raw.githubusercontent.com/PacktWorkshops/The-Applied-Artificial-Intelligence-Workshop/master/Datasets/Sales_Transactions_Dataset_Weekly.csv"
df = pd.read_csv(file_url)

# Preview the first few rows
df.head()


,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,Normalized 42,Normalized 43,Normalized 44,Normalized 45,Normalized 46,Normalized 47,Normalized 48,Normalized 49,Normalized 50,Normalized 51
0,P1,11,12,10,8,13,12,14,21,6,...,0.06,0.22,0.28,0.39,0.50,0.00,0.22,0.17,0.11,0.39
1,P2,7,6,3,2,7,1,6,3,3,...,0.20,0.40,0.50,0.10,0.10,0.40,0.50,0.10,0.60,0.00
2,P3,7,11,8,9,10,8,7,13,12,...,0.27,1.00,0.18,0.18,0.36,0.45,1.00,0.45,0.45,0.36
3,P4,12,8,13,5,9,6,9,13,13,...,0.41,0.47,0.06,0.12,0.24,0.35,0.71,0.35,0.29,0.35
4,P5,8,5,13,11,6,7,9,14,9,...,0.27,0.53,0.27,0.60,0.20,0.20,0.13,0.53,0.33,0.40


In [7]:
#Preparing Features for K-Means
"""
K-Means clustering will be performed on the **normalized weekly sales columns** (`Normalized 0` to `Normalized 51`).  
This ensures that the clustering is based on sales patterns rather than raw scale differences.
"""
# Keep only normalized columns for clustering
df2 = df.drop(df.iloc[:, 0:55], inplace=False, axis=1)

#Confirm noramlzied weekly features
df2.shape


(811, 52)

In [8]:
from sklearn.cluster import KMeans

# Train K-Means clustering model
k_means_model = KMeans(n_clusters=8, random_state=8)
k_means_model.fit(df2)

# Cluster labels for each product
labels = k_means_model.labels_
labels[:20]


array([6, 6, 2, 6, 6, 1, 6, 6, 2, 2, 6, 2, 5, 2, 5, 5, 5, 5, 5, 5],
      dtype=int32)

In [4]:
# Keep only weekly sales columns (W0 to W51), drop normalized columns and Product_Code
df_sales = df.copy()
df_sales.drop(df_sales.iloc[:, 53:], inplace=True, axis=1)  # drop normalized columns
df_sales.drop("Product_Code", inplace=True, axis=1)

# Add cluster label
df_sales["label"] = labels

df_sales.head()


,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W43,W44,W45,W46,W47,W48,W49,W50,W51,label
0,11,12,10,8,13,12,14,21,6,14,...,7,8,10,12,3,7,6,5,10,6
1,7,6,3,2,7,1,6,3,3,3,...,4,5,1,1,4,5,1,6,0,6
2,7,11,8,9,10,8,7,13,12,6,...,14,5,5,7,8,14,8,8,7,2
3,12,8,13,5,9,6,9,13,13,11,...,10,3,4,6,8,14,8,7,8,6
4,8,5,13,11,6,7,9,14,9,9,...,11,7,12,6,6,5,11,8,9,6


In [9]:
# Total weekly sales per cluster
df_agg = df_sales.groupby("label").sum()

# Number of products per cluster
df_final = df_sales[["label", "W0"]].groupby("label").count().rename(columns={"W0": "count_product"})

# Total and average yearly sales per cluster
df_final["total_sales"] = df_agg.sum(axis=1)
df_final["yearly_average_sales"] = df_final["total_sales"] / df_final["count_product"]

# Sort clusters by average yearly sales
df_final.sort_values(by="yearly_average_sales", ascending=False, inplace=True)

df_final


,count_product,total_sales,yearly_average_sales
label,,,
5,128,173808,1357.875000
6,129,86341,669.310078
2,96,48778,508.104167
1,90,43112,479.022222
4,85,17390,204.588235
7,107,4348,40.635514
0,8,96,12.000000
3,168,1414,8.416667
